In [4]:
import tkinter as tk    #for gui
from tkinter import scrolledtext   #for scrollable text as in real chat
import nltk   #for language processing
import numpy as np  #for scientific calculations
import json  #for importing the intents json file
import random  #for generating a random response from a list of responses
from keras.models import Sequential #sequential is used to define the architecture
from keras.layers import Dense, Embedding, LSTM, Dropout
#dense connects the layers, embedding is used for embedding words, LSTM is used for handling sequential data,
#dropout is to prevent overfitting(the model fits exactly against its training data)
from nltk.stem import WordNetLemmatizer #this is used to reduce the words to their root form(also called tokenization)
#we will use it to lemmatize the words obtained from the intents.json dataset

# Download NLTK data (if not already downloaded)
# nltk.download('punkt')
# nltk.download('wordnet')

# Load intents file
with open('intents.json', 'r') as file:  #opening the json file in read-only mode
    intents = json.load(file)

# Extract words and classes from intents file
words = []
classes = []
documents = []
ignore_words = ['?', '!']
lemmatizer = WordNetLemmatizer()

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize words
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize words and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

# Build the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model to the training data
model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)

# Function to clean and tokenize user input
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Function to create a bag of words from user input
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"Found in bag: {w}")
    return np.array(bag)

# Function to predict the class of the user input and get a response
def predict_class(sentence):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# Function to handle user input and generate a response
def get_response():
    user_input = input_box.get("1.0", tk.END)
    response = "I'm sorry, I don't understand."
    ints = predict_class(user_input)
    tag = ints[0]['intent']
    for intent in intents['intents']:
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])
            break
    chat_history.config(state=tk.NORMAL)
    chat_history.insert(tk.END, f"You: {user_input}\n", "user")
    chat_history.insert(tk.END, f"Purple: {response}\n", "bot")
    chat_history.config(state=tk.DISABLED)

    # Clear the input box
    input_box.delete("1.0", tk.END)
    
# Create the main window
root = tk.Tk()
root.title("TriumphTidings")
root.configure(bg="black")

# Create a scrolled text widget to display chat history
chat_history = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=50, height=20,bg="#87ceeb",fg="black",font=("Billabong",11,"bold","italic"))
chat_history.tag_configure("user", justify="right", foreground="blue")#aligning user's text to right
chat_history.tag_configure("bot", justify="left", foreground="black")#aligning bot's text to left
chat_history.grid(row = 0, column = 0,padx=10, pady=10)

# Create an input box for user input
input_box = tk.Text(root, wrap=tk.WORD, width=50, height=2,font=("Billabong",11))
input_box.grid(row = 1, column = 0, padx=10, pady=10)

# Create a button to send user input and get a response
send_button = tk.Button(root, text="Send", command=get_response,bg="#FFFF00",font=("Billabong",11,"bold","italic"))
send_button.grid(row = 1, column = 1,padx=5, pady=10)

# Start the tkinter main loop
root.mainloop()


Epoch 1/100
13/13 [==============================] - 1s 3ms/step - loss: 2.4909 - accuracy: 0.0625
Epoch 2/100
13/13 [==============================] - 0s 3ms/step - loss: 2.4557 - accuracy: 0.0781
Epoch 3/100
13/13 [==============================] - 0s 3ms/step - loss: 2.4176 - accuracy: 0.1250
Epoch 4/100
13/13 [==============================] - 0s 3ms/step - loss: 2.3555 - accuracy: 0.2344
Epoch 5/100
13/13 [==============================] - 0s 3ms/step - loss: 2.3193 - accuracy: 0.2188
Epoch 6/100
13/13 [==============================] - 0s 3ms/step - loss: 2.2977 - accuracy: 0.2812
Epoch 7/100
13/13 [==============================] - 0s 3ms/step - loss: 2.2118 - accuracy: 0.2656
Epoch 8/100
13/13 [==============================] - 0s 3ms/step - loss: 2.1543 - accuracy: 0.3125
Epoch 9/100
13/13 [==============================] - 0s 3ms/step - loss: 2.1268 - accuracy: 0.4219
Epoch 10/100
13/13 [==============================] - 0s 3ms/step - loss: 2.1086 - accuracy: 0.4062
Epoch 11/

13/13 [==============================] - 0s 3ms/step - loss: 0.2036 - accuracy: 0.9688
Epoch 84/100
13/13 [==============================] - 0s 2ms/step - loss: 0.1037 - accuracy: 0.9688
Epoch 85/100
13/13 [==============================] - 0s 2ms/step - loss: 0.1498 - accuracy: 1.0000
Epoch 86/100
13/13 [==============================] - 0s 2ms/step - loss: 0.1257 - accuracy: 0.9844
Epoch 87/100
13/13 [==============================] - 0s 2ms/step - loss: 0.1345 - accuracy: 0.9531
Epoch 88/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0881 - accuracy: 0.9844
Epoch 89/100
13/13 [==============================] - 0s 2ms/step - loss: 0.1044 - accuracy: 0.9844
Epoch 90/100
13/13 [==============================] - 0s 2ms/step - loss: 0.1358 - accuracy: 0.9688
Epoch 91/100
13/13 [==============================] - 0s 2ms/step - loss: 0.1539 - accuracy: 0.9531
Epoch 92/100
13/13 [==============================] - 0s 2ms/step - loss: 0.1145 - accuracy: 0.9844
Epoch 93/100
